# COLLATING DATA

In [1]:
import sqlalchemy
import psycopg2
import pandas as pd

# DSN (data source name) format for database connections:  
# [protocol / database  name]://[username]:[password]@[hostname / ip]:[port]/[database name here]


# on your computer you are the user postgres (full administrative access)
db_user = 'postgres'
# if you need a password to access a database, put it here
db_password = ''
# on your computer, use localhost
db_host = 'localhost'
# the default port for postgres is 5432
db_port = 5432
# we want to creat a database with the following name
database = 'final_db'
# #set up a connection to postgres without specifying the database
# conn_postgres = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}'

In [2]:
# create an sqlalchemy connection
engine = sqlalchemy.engine.create_engine(conn_postgres)

# connect using sqlalchemy
connection = engine.connect()

In [3]:
# # create database
# connection.execute(f"create database {database}")

In [4]:
# close connection
connection.close()

NameError: name 'connection' is not defined

In [5]:
# open a new connection to the database that we created
conn_str = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{database}'
conn = psycopg2.connect(conn_str)

# connect using sqlalchemy
engine = sqlalchemy.engine.create_engine(conn_str)

# connect using sqlalchemy
connection = engine.connect()

In [ ]:
# data = pd.read_csv('../.csv')
# data.drop(columns = 'Unnamed: 0', inplace=True)
# data.columns = ['{}_{}'.format('br', col.lower()) for col in data.columns]
# data.columns = ['{}_{}'.format('nba', col.lower()) for col in data.columns]


# # def min_converter(x):
#     try:
#         y = x.split(':')
#         y_secs = int(y[0])*60 + int(y[1])
#         return y_secs
#     except:
#         return 0

# data.nba_min = data.nba_min.apply(min_converter)


# data.nba_game_id = data.nba_game_id.apply(lambda x: str(x))
# data.nba_team_id = data.nba_team_id.apply(lambda x: str(x))
# data.nba_season_id = data.nba_season_id.apply(lambda x: str(x))
# data.nba_person_id = data.nba_person_id.apply(lambda x: str(x))
# data.nba_height = data.nba_height.apply(ht_converter)
# data.nba_weight = data.nba_weight.apply(ht_converter)
# data.nba_birthdate = data.nba_birthdate.apply(lambda x: x[:10])
# data.nba_from_year = data.nba_from_year.apply(lambda x: int(x))
# data.nba_from_year = data.nba_from_year.apply(lambda x: int(x))

# import unidecode

# data['br_name'] = data['br_name'].apply(lambda x: unidecode.unidecode(x))
# data['br_location'] = data['br_location'].apply(lambda x: x.replace('Location.', ''))
# data['br_outcome'] = data['br_outcome'].apply(lambda x: x.replace('Outcome.','')[0])
# data['br_team'] = data['br_team'].map(lambda x: abbr_dict[x.replace('Team.','').replace('_', ' ')])
# data['br_opponent'] = data['br_opponent'].map(lambda x: abbr_dict[x.replace('Team.','').replace('_', ' ')])
# data['br_slug'] = data['br_slug'].map(lambda x: key_changer[x] if key_changer[x]!='None' else x)

# print(data.info())
# data.head()

# data.to_sql(name = '...', con = engine, if_exists = 'replace', index = False)

In [6]:
query = """
SELECT DISTINCT(a.br_player), b.nba_display_first_last, b.nba_person_id, a.br_birth_date, c.br_slug
FROM br_attrs_peryear a
LEFT JOIN(
SELECT REPLACE(nba_display_first_last, ' Jr.', '') nba_player, nba_display_first_last, nba_person_id, nba_birthdate
FROM nba_common_attr) b
ON (a.br_birth_date = b.nba_birthdate AND 
REPLACE(a.br_player, ' Jr.','') = b.nba_player)
OR (a.br_birth_date = b.nba_birthdate AND 
REPLACE(a.br_player, '.','') = REPLACE(b.nba_display_first_last, '.',''))
OR (a.br_birth_date = b.nba_birthdate AND 
REPLACE(a.br_player, '.','') = REPLACE(b.nba_display_first_last, ' III',''))
OR (a.br_birth_date = b.nba_birthdate AND 
REPLACE(a.br_player, '.','') = REPLACE(b.nba_display_first_last, ' II',''))
LEFT JOIN(
SELECT DISTINCT(br_slug), br_name
FROM br_tra_box) c
ON (REPLACE(a.br_player, ' Jr.','') = REPLACE(c.br_name, ' Jr.',''))
"""

df = pd.read_sql(query, con=conn)

In [15]:
df[(df['br_player'].duplicated(keep=False))&
   (df['br_birth_date'].duplicated(keep=False))].sort_values(by=['br_player','br_birth_date'])

,br_player,nba_display_first_last,nba_person_id,br_birth_date,br_slug
936,Chris Johnson,None,None,1985-07-15,johnsch03
1979,Chris Johnson,None,None,1985-07-15,johnsch04
38,Chris Johnson,Chris Johnson,203187,1990-04-29,johnsch04
1641,Chris Johnson,Chris Johnson,203187,1990-04-29,johnsch03
610,Chris Wright,Chris Wright,202874,1988-09-30,wrighch02
921,Chris Wright,Chris Wright,202874,1988-09-30,wrighch01
185,Chris Wright,Chris Wright,203203,1989-11-04,wrighch01
589,Chris Wright,Chris Wright,203203,1989-11-04,wrighch02
1056,Dee Brown,Dee Brown,244,1968-11-29,brownde01
1526,Dee Brown,Dee Brown,244,1968-11-29,brownde03


In [18]:
duplicates = [1979,1641,610,185,1526,719,1609,1294,2020,985,1614,1904,
             351,1068,1640,1961,1817,1848,1115,1264,1597,1731,210,1560,
             235,1835,1419,1506,1678,277,1169,702]

df.drop(duplicates, inplace=True)

In [45]:
df.drop(df[df.nba_person_id.isna()].index, inplace=True)

In [52]:
df[df.nba_person_id.duplicated(keep=False)].sort_values(by='br_slug')

,br_player,nba_display_first_last,nba_person_id,br_birth_date,br_slug


In [48]:
duplicates_cont = [223,44,245,1707,1635,1954,247,37,584]

df.drop(duplicates_cont, inplace=True)

In [56]:
dict(zip(df.br_slug, df.nba_person_id))

{'mccreer01': '1628571',
 'douglch01': '201604',
 'poythal01': '1627816',
 'arthuda01': '201589',
 'harpero01': '166',
 'fosteje01': '1902',
 'frankja01': '203479',
 'jacksji01': '754',
 'hawessp01': '201150',
 'trepaje01': '2244',
 'gibsota01': '201959',
 'hughela01': '1716',
 'knighbr01': '1510',
 'lasmest01': '201186',
 'bateske01': '1628966',
 'rosede01': '201565',
 'mccalta01': '1628769',
 'gardnth01': '201242',
 'siakapa01': '1627783',
 'simiewa01': '101134',
 'maynoer01': '201953',
 'garrebi01': '1628656',
 'hansbty01': '201946',
 'cunnija01': '203099',
 'divindo01': '1628978',
 'jonesbo02': '200784',
 'johnsan02': '1533',
 'davisem01': '1023',
 'jeffrda01': '1629610',
 'corbity01': '895',
 'coffeam01': '1629599',
 'samakso01': '2066',
 'nailole01': '1924',
 'johnsch04': '203187',
 'ewingpa02': '201607',
 'satorto01': '203107',
 'sallejo01': '927',
 'powelka01': '2867',
 'akognjo01': '203006',
 'kasunma01': '2435',
 'jeromty01': '1629660',
 'breweja01': '2249',
 'engliki01': '20

In [57]:
query = """
SELECT *
FROM br_tra_box
"""

data = pd.read_sql(query, con=conn)

data.br_slug = data.br_slug.map(player_keys)

# data.to_sql(name = '...', con = engine, if_exists = 'replace', index = False)

In [68]:
checking_list = list(data[data.br_slug.isna()]['br_name'].unique())

In [70]:
df[df['br_player'].isin(checking_list)]

,br_player,nba_display_first_last,nba_person_id,br_birth_date,br_slug
38,Chris Johnson,Chris Johnson,203187,1990-04-29,johnsch04


In [38]:
len(player_keys), len(df)

(2033, 2118)

In [32]:
data

,br_slug,br_name,br_team,br_location,br_opponent,br_outcome,br_seconds_played,br_made_field_goals,br_attempted_field_goals,br_made_three_point_field_goals,...,br_attempted_free_throws,br_offensive_rebounds,br_defensive_rebounds,br_assists,br_steals,br_blocks,br_turnovers,br_personal_fouls,br_game_score,br_date
0,1628571,Sam Cassell,MIL,AWAY,HOU,W,2250,15,20,0,...,7,1,2,11,1,0,4,5,30.2,1999-11-2
1,201604,Nick Van Exel,DEN,HOME,PHO,W,2733,12,21,5,...,7,2,7,9,0,0,3,1,29.7,1999-11-2
2,1627816,Paul Pierce,BOS,AWAY,TOR,W,2273,12,19,3,...,6,1,7,5,2,0,2,3,25.4,1999-11-2
3,201589,Lamar Odom,LAC,HOME,SEA,L,2607,10,18,2,...,15,2,10,3,2,2,3,1,25.1,1999-11-2
4,166,Grant Hill,DET,AWAY,MIA,L,3134,16,35,1,...,11,3,6,5,0,1,4,5,23.8,1999-11-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
554660,NaN,Derrick Jones,MIA,HOME,LAL,L,87,0,0,0,...,0,0,0,0,0,0,0,0,0.0,2020-10-11
554661,NaN,Quinn Cook,LAL,AWAY,MIA,W,87,0,1,0,...,0,0,0,0,0,0,0,0,-0.7,2020-10-11
554662,NaN,Jared Dudley,LAL,AWAY,MIA,W,87,0,1,0,...,0,0,0,0,0,0,0,0,-0.7,2020-10-11
554663,NaN,Andre Iguodala,MIA,HOME,LAL,L,633,0,2,0,...,0,0,1,0,0,0,0,1,-1.5,2020-10-11


In [ ]:
conn.commit()

In [ ]:
# close connection
connection.close()